In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import os

## An irrelevant test to check whether pickle is better than csv 

In [2]:
trial_pkl =  pd.read_pickle(
    '/mnt/dzl_bioinf/binliu/deepRNA/data_all_samples/all_samples_community_level.pkl')


In [3]:
trial_csv = trial_pkl.to_csv('/mnt/dzl_bioinf/binliu/deepRNA/data_all_samples/all_samples_transcript_level_csv_format.csv')


In [23]:
import os
file_stat_pkl = os.stat(
    '/mnt/dzl_bioinf/binliu/deepRNA/data_all_samples/all_samples_community_level.pkl').st_size
file_stat_csv = os.stat(
    '/mnt/dzl_bioinf/binliu/deepRNA/data_all_samples/all_samples_transcript_level_csv_format.csv').st_size
print("File size of pkl file is {} MB, that of the csv file is {} MB".format(
    round(file_stat_pkl/(1024*1024)), round(file_stat_csv/(1024*1024))))


File size of pkl file is 433 MB, that of the csv file is 949 MB


In [5]:
trial_csv = pd.read_csv(
    '/mnt/dzl_bioinf/binliu/deepRNA/data_all_samples/all_samples_transcript_level_csv_format.csv')


Pickle is much faster than csv and uses up smaller space on the disk. 

## Train Test split for all related results

In [6]:
annotate_tem = pd.read_excel(
    "/mnt/dzl_bioinf/binliu/EMBL_ExpressionArray/arrayExpress_annotation.xlsx")

In [7]:
annotate_tem['Characteristics[organism part]']

0               lung
1              ovary
2              colon
3              colon
4               lung
            ...     
27882          brain
27883          brain
27884          brain
27885    bone marrow
27886    bone marrow
Name: Characteristics[organism part], Length: 27887, dtype: object

In [8]:
organism = annotate_tem['Characteristics[organism part]']
removed_ids = annotate_tem['Source Name'][organism == '  ']


In [9]:
len(removed_ids)
# There are a few samples with missing organsisms, but maybe we can still use them for the training and testing pipeline. 

5163

In [10]:
ori_pickle_trans = pd.read_pickle(
    '/mnt/dzl_bioinf/binliu/deepRNA/data_all_samples/all_samples_transcript_level.pkl')


In [11]:
ori_pickle_trans = ori_pickle_trans.astype(float)

In [12]:
ori_pickle_trans.shape

(27887, 54675)

In [13]:
sum(annotate_tem['Source Name'] != ori_pickle_trans.index)

0

In [14]:
organism_dis = organism.value_counts().to_dict()


In [15]:
removed_organism = []

In [16]:
for key in organism_dis:
    if organism_dis[key]  == 1:
        removed_organism.append(key)

In [17]:
len(removed_organism)

24

In [18]:
removed_ids = [row['Source Name']
               for _, row in annotate_tem.iterrows() if row['Characteristics[organism part]'] in removed_organism]
new_organism = [row['Characteristics[organism part]']
                for _, row in annotate_tem.iterrows() if row['Characteristics[organism part]'] not in removed_organism]


In [19]:
ori_pickle_trans = ori_pickle_trans[~ori_pickle_trans.index.isin(removed_ids)]


In [20]:
ori_pickle_trans.shape

(27863, 54675)

In [21]:
df_train_trans,  df_test_trans, train_tem_label,  test_tem_label = train_test_split(
    ori_pickle_trans, new_organism, train_size=0.90, random_state=42, stratify=new_organism)


In [22]:
print(df_train_trans.shape)
print(df_test_trans.shape)

(25076, 54675)
(2787, 54675)


In [24]:
df_train_trans.to_pickle(
    '/mnt/dzl_bioinf/binliu/deepRNA/data_all_samples/transcript_level_train_test/all_samples_transcript_level_train.pkl')
df_test_trans.to_pickle(
    '/mnt/dzl_bioinf/binliu/deepRNA/data_all_samples/transcript_level_train_test/all_samples_transcript_level_test.pkl')


In [25]:
train_index = df_train_trans.index
test_index = df_test_trans.index

In [26]:
ori_pickle_gene = pd.read_pickle(
    '/mnt/dzl_bioinf/binliu/deepRNA/data_all_samples/all_samples_gene_level.pkl')


In [31]:
df_train_genes = ori_pickle_gene.loc[train_index]
df_test_genes = ori_pickle_gene.loc[test_index]

In [32]:
df_train_genes.index[0:5]

Index(['Sample 10832', 'Sample 17689', 'Sample 25789', 'Sample 6156',
       'Sample 15773'],
      dtype='object', name='CompositeSequence Identifier')

In [33]:
df_train_genes.to_pickle(
    '/mnt/dzl_bioinf/binliu/deepRNA/data_all_samples/gene_level_train_test/all_samples_gene_level_train.pkl')
df_test_genes.to_pickle(
    '/mnt/dzl_bioinf/binliu/deepRNA/data_all_samples/gene_level_train_test/all_samples_gene_level_test.pkl')


In [34]:
ori_pickle_community = pd.read_pickle(
    '/mnt/dzl_bioinf/binliu/deepRNA/data_all_samples/all_samples_community_level.pkl')


In [35]:
df_train_communities = ori_pickle_community.loc[train_index]
df_test_communities = ori_pickle_community.loc[test_index]


In [36]:
df_train_communities.to_pickle(
    '/mnt/dzl_bioinf/binliu/deepRNA/data_all_samples/community_level_train_test/all_samples_community_level_train.pkl')
df_test_communities.to_pickle(
    '/mnt/dzl_bioinf/binliu/deepRNA/data_all_samples/community_level_train_test/all_samples_community_level_test.pkl')
